In [ ]:
# Khóa chính của Bảng StorageRecords_Raw là record_id vì: 
# Mỗi record_id xác định duy nhất một bản ghi trong bảng
# Không có giá trị record_id nào bị trùng lặp/NULL

In [ ]:
# Bảng chưa đạt chuẩn 1NF: Các cột có kiểu dữ liệu nhất quán và có tên duy nhất
# Bị lặp lại thông tin (tại cột warehouse_id, warehouse_name lặp lại nhiều lần: cặp WH01-Main Warehouse lặp lại 3 lần)

In [ ]:
# 2F:
# Bảng có các thuộc tính không phụ thuộc đầy đủ vào khóa chính (record_id)
#     warehouse_name (phụ thuộc vào warehouse_id)
# customer_name (phụ thuộc vào customer_id)
#    staff_name (phụ thuộc vào staff_id)
#    item_name (phụ thuộc vào item_code)

In [ ]:
# 3F:
# Có các thuộc tính không khóa phụ thuộc vào thuộc tính không khóa khác

In [ ]:
!pip install mysql-connector-python

In [ ]:
# import mysql.connector
# db = mysql.connector.connect(user = "root", password = "abc13579",
#                              host="localhost")

# print(db)

# query = 'CREATE SCHEMA database_d43'
# cursor = db.cursor()
# cursor.execute(query)

In [5]:
import mysql.connector 

connection = mysql.connector.connect(
    host="127.0.0.1",     
    user="root",         
    password="2011DongBaLiem@",
    database="chuanthu3")
cursor = connection.cursor()

In [4]:
query="""CREATE SCHEMA chuanthu3"""
cursor.execute(query)

In [8]:
cursor.execute("""
create table if not exists chuanthu3.Items(
    item_code VARCHAR(10) PRIMARY KEY,
    item_name VARCHAR(50)
) ENGINE=InnoDB;
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS Warehouses(
    warehouse_id VARCHAR(10) PRIMARY KEY,
    warehouse_name VARCHAR(50)
) ENGINE=InnoDB;
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS Customers(
    customer_id VARCHAR(10) PRIMARY KEY,
    customer_name VARCHAR(50)
) ENGINE=InnoDB;
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Staffs(
    staff_id VARCHAR(10) PRIMARY KEY,
    staff_name VARCHAR(50)
) ENGINE=InnoDB;
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS StorageRecord(
    record_id VARCHAR(10) PRIMARY KEY,
    item_code VARCHAR(10),
    quantity INT,
    warehouse_id VARCHAR(10),
    location VARCHAR(50),
    customer_id VARCHAR(10),
    staff_id VARCHAR(10),
    FOREIGN KEY(item_code) REFERENCES Items(item_code),
    FOREIGN KEY(warehouse_id) REFERENCES Warehouses(warehouse_id),
    FOREIGN KEY(customer_id ) REFERENCES Customers(customer_id),
    FOREIGN KEY(staff_id) REFERENCES Staffs(staff_id)
) ENGINE=InnoDB;
""")
connection.commit()

In [12]:
cursor.execute("""
INSERT IGNORE INTO Items(item_code, item_name) VALUES
    ('IT001', 'Motor Oil 5L'),
    ('IT002', 'Engine Cleaner'),
    ('IT003', 'Battery 12V')
;
""")

cursor.execute("""
INSERT IGNORE INTO Warehouses VALUES
    ('WH01', 'Main Warehouse'),
    ('WH02', 'North Depot')
;
""")

cursor.execute("""
INSERT IGNORE INTO Customers VALUES
    ('C001', 'Mekong Corp'),
    ('C002', 'Vina Auto'),
    ('C003', 'Delta Co')
;
""")

cursor.execute("""
INSERT IGNORE INTO Staffs VALUES
    ('S01', 'Hoa Nguyen'),
    ('S02', 'Khang Tran')
;
""")

cursor.execute("""
INSERT IGNORE INTO StorageRecord VALUES
    ('R001', 'IT001', 50, 'WH01', 'Shelf A1', 'C001', 'S01'),
    ('R002', 'IT002', 30, 'WH01', 'Shelf B2', 'C002', 'S02'),
    ('R003', 'IT001', 40, 'WH02', 'Shelf C3', 'C001', 'S01'),
    ('R004', 'IT003', 25, 'WH01', 'Shelf A1', 'C003', 'S02')    
;
""")
connection.commit()

In [13]:
import pandas as pd

In [14]:
query="""
SELECT * 
FROM StorageRecord
;
"""
connection.commit()
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_26428\1132945239.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,record_id,item_code,quantity,warehouse_id,location,customer_id,staff_id
0,R001,IT001,50,WH01,Shelf A1,C001,S01
1,R002,IT002,30,WH01,Shelf B2,C002,S02
2,R003,IT001,40,WH02,Shelf C3,C001,S01
3,R004,IT003,25,WH01,Shelf A1,C003,S02


In [16]:
query="""
SELECT w.warehouse_name, i.item_name, SUM(sr.quantity) AS total_quantity
FROM StorageRecord sr
JOIN Items i ON sr.item_code = i.item_code
JOIN Warehouses w ON sr.warehouse_id = w.warehouse_id
GROUP BY w.warehouse_name, i.item_name
;
"""
connection.commit()
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_26428\3392769882.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,warehouse_name,item_name,total_quantity
0,Main Warehouse,Motor Oil 5L,50.0
1,North Depot,Motor Oil 5L,40.0
2,Main Warehouse,Engine Cleaner,30.0
3,Main Warehouse,Battery 12V,25.0


In [18]:
query="""
SELECT DISTINCT c.customer_name, i.item_name
FROM StorageRecord sr
JOIN Items i ON sr.item_code = i.item_code
JOIN Customers c ON c.customer_id = sr.customer_id
;
"""
connection.commit()
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_26428\853886102.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,customer_name,item_name
0,Mekong Corp,Motor Oil 5L
1,Vina Auto,Engine Cleaner
2,Delta Co,Battery 12V


In [20]:
query="""
SELECT DISTINCT s.staff_name
FROM StorageRecord sr
JOIN Staffs s ON s.staff_id = sr.staff_id
JOIN Warehouses w ON w.warehouse_id = sr.warehouse_id
WHERE w.warehouse_name = 'Main Warehouse'
;
"""
connection.commit()
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_26428\1095766047.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,staff_name
0,Hoa Nguyen
1,Khang Tran


In [22]:

query="""
SELECT DISTINCT w.warehouse_name, i.item_name, sr.location
FROM StorageRecord sr
JOIN Items i ON i.item_code = sr.item_code
JOIN Warehouses w ON w.warehouse_id = sr.warehouse_id
;
"""

connection.commit()
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_26428\2433843243.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,warehouse_name,item_name,location
0,Main Warehouse,Motor Oil 5L,Shelf A1
1,North Depot,Motor Oil 5L,Shelf C3
2,Main Warehouse,Engine Cleaner,Shelf B2
3,Main Warehouse,Battery 12V,Shelf A1


In [24]:
query="""
SELECT 
    w.warehouse_name,
    COUNT(DISTINCT sr.customer_id) AS total_customers
FROM StorageRecord sr
JOIN Customers c ON c.customer_id = sr.customer_id
JOIN Warehouses w ON w.warehouse_id = sr.warehouse_id
GROUP BY w.warehouse_name
;
"""
connection.commit()
data=pd.read_sql_query(query, connection)
data

# GROUP BY yêu cầu rằng tất cả các trường trong SELECT phải hoặc có mặt trong GROUP BY, hoặc phải được tính bằng hàm tổng hợp

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_26428\3865712710.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,warehouse_name,total_customers
0,Main Warehouse,3
1,North Depot,1


In [25]:
query="""
SELECT 
    w.warehouse_name,
    GROUP_CONCAT(DISTINCT c.customer_name) AS customer_names,
    COUNT(DISTINCT sr.customer_id) AS total_customers
FROM StorageRecord sr
JOIN Customers c ON c.customer_id = sr.customer_id
JOIN Warehouses w ON w.warehouse_id = sr.warehouse_id
GROUP BY w.warehouse_name;
"""
connection.commit()
data=pd.read_sql_query(query, connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_26428\66703259.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query, connection)


,warehouse_name,customer_names,total_customers
0,Main Warehouse,"Delta Co,Mekong Corp,Vina Auto",3
1,North Depot,Mekong Corp,1
